In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
0,Зарубежные фильмы,26 Май 23,Заклятье. Первое пробуждение / La Niña de la Comunión / The Communion Girl (2022) BDRip-AVC от DoMiNo & селезень | D,1.68 GB,44,21
6,Зарубежные фильмы,26 Май 23,Заклятье. Первое пробуждение / La Niña de la Comunión / The Communion Girl (2022) BDRip 1080p от селезень | D,8.37 GB,0,75
12,Зарубежные фильмы,26 Май 23,Люби снова / Love Again (2023) WEB-DL 1080p от New-Team | Jaskier,6.61 GB,48,56
16,Зарубежные фильмы,26 Май 23,Обслуживание прекращено / Discontinued (2022) WEB-DLRip-AVC от DoMiNo & селезень | P | TVShows,1.09 GB,51,24
18,Зарубежные фильмы,26 Май 23,Обслуживание прекращено / Discontinued (2022) WEB-DLRip 720p от ExKinoRay | TVShows,1.61 GB,45,13
23,Зарубежные фильмы,26 Май 23,"Джон Уик 4 / John Wick: Chapter 4 (2023) WEB-DL 2160p | 4K | SDR | D, P | Line",19.63 GB,134,183
24,Зарубежные фильмы,26 Май 23,Обслуживание прекращено / Discontinued (2022) WEB-DLRip | TVShows,1.46 GB,115,23
30,Зарубежные фильмы,26 Май 23,Обслуживание прекращено / Discontinued (2022) WEB-DL 1080p от New-Team | TVShows,2.72 GB,267,51
32,Зарубежные фильмы,26 Май 23,"Джон Уик 4 / John Wick: Chapter 4 (2023) WEB-DL 1080p от селезень | P, A",12.28 GB,340,512
33,Зарубежные фильмы,26 Май 23,Подземелья и драконы: Честь среди воров / Dungeons Dragons: Honor Among Thieves (2023) BDRip 1080p от ExKinoRay | NewComers,14.34 GB,25,41


In [7]:
id = [19, 76]
my_df.loc[id, 'href']

KeyError: "None of [Int64Index([19, 76], dtype='int64')] are in the [index]"